In [48]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.api import stats


prices_1 = pd.read_csv('../data/round-4-island-data-bottle/prices_round_4_day_1.csv', sep=';')
prices_2 = pd.read_csv('../data/round-4-island-data-bottle/prices_round_4_day_2.csv', sep=';')
prices_3 = pd.read_csv('../data/round-4-island-data-bottle/prices_round_4_day_3.csv', sep=';')

In [49]:
timeseries = pd.concat([prices_1, prices_2,prices_3])
timeseries_coconut = timeseries[timeseries['product'].isin(['COCONUT'])][["mid_price"]]
timeseries_coupon = timeseries[timeseries['product'].isin(['COCONUT_COUPON'])][["mid_price"]]

In [51]:
# fit 20 step arima model to timeseries coconut and add predictions to dataframe
from statsmodels.tsa.arima.model import ARIMA

arima_model = ARIMA(timeseries_coconut, order=(20,1,0))
arima_model_fit = arima_model.fit()


# Add predictions to dataframe as a new column
timeseries_coconut['arima'] = arima_model_fit.predict(start=0, end=len(timeseries_coconut)-1)

# Benchmark against last price
timeseries_coconut['last_price'] = timeseries_coconut['mid_price'].shift(1)

#Benchmark against 20 day moving average
timeseries_coconut['moving_average'] = timeseries_coconut['mid_price'].rolling(window=20).mean()

# calculate mse between actual and each predicted price
timeseries_coconut['mse_arima'] = (timeseries_coconut['mid_price'] - timeseries_coconut['arima'])**2
timeseries_coconut['mse_last_price'] = (timeseries_coconut['mid_price'] - timeseries_coconut['last_price'])**2
timeseries_coconut['mse_moving_average'] = (timeseries_coconut['mid_price'] - timeseries_coconut['moving_average'])**2

/Users/michaelfuest/.pyenv/versions/3.12.2/envs/prosperity/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/michaelfuest/.pyenv/versions/3.12.2/envs/prosperity/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/Users/michaelfuest/.pyenv/versions/3.12.2/envs/prosperity/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


KeyError: '[0] not found in axis'

In [54]:
# calculate mean squared error and print each one
mse_arima = timeseries_coconut['mse_arima'].mean()
mse_last_price = timeseries_coconut['mse_last_price'].mean()
mse_moving_average = timeseries_coconut['mse_moving_average'].mean()

print(f"Mean Squared Error ARIMA: {mse_arima}")
print(f"Mean Squared Error Last Price: {mse_last_price}")
print(f"Mean Squared Error Moving Average: {mse_moving_average}")

Mean Squared Error ARIMA: 1.058040849368793
Mean Squared Error Last Price: 1.0596726339300597
Mean Squared Error Moving Average: 6.39158302478403
